<font color=#163e5a>  
    <h1>Preprocessing the data</h1>
    <p>The following command in jupyter notebook works ! =)</p>
</font>

In [1]:
!python D:/Documents/PythonScripts/PhDScripts/Pt_p2/preprocess_bcdi.py

homedir : D:/Documents/PythonScripts/PhDScripts/Pt_p2/
root_folder : D:/Documents/PythonScripts/PhDScripts/Pt_p2/
File has time stamps issues
File has time stamps issues

Scan 1398
Setup:  SIXS_2019
Detector:  Maxipix
Pixel number (VxH):  516 516
Detector ROI: [0, 303, 0, 296]
Horizontal pixel size with binning:  5.5e-05 m
Vertical pixel size with binning:  5.5e-05 m
Specfile:  D:/Documents/PythonScripts/PhDScripts/Pt_p2/analysis/alias_dict.txt
Scan type:  inplane
Output will be non orthogonal, in the detector frame
Detector ROI loaded (VxH): 303 296
Detector physical size without binning (VxH): 516 516
Detector size with binning (VxH): 516 516

Loading frame 1
Loading frame 2
Loading frame 3
Loading frame 4
Loading frame 5
Loading frame 6
Loading frame 7
Loading frame 8
Loading frame 9
Loading frame 10
Loading frame 11
Loading frame 12
Loading frame 13
Loading frame 14
Loading frame 15
Loading frame 16
Loading frame 17
Loading frame 18
Loading frame 19
Loading frame 20
Loading frame 2

<font color=#163e5a>
    <h1>Preprocess output</h1>


**Two .npz files**
* <a href = "https://numpy.org/doc/stable/reference/generated/numpy.savez.html" target = "_blank">Documentation</a>
* the mask: S1398_maskpynx_norm_128_300_294_1_1_1.npz
* the (normalised?) detector data: S1398_pynx_norm_128_300_294_1_1_1.npz
    
<h2>Find the .npz files</h2>
</font>

In [1]:
%%bash
ls Pt_p2/pynxraw/*.npz

Pt_p2/pynxraw/Mask_10_100_0_1.npz
Pt_p2/pynxraw/Mask_Threshold_0.15.npz
Pt_p2/pynxraw/Mask_Threshold_0.2.npz
Pt_p2/pynxraw/Mask_Threshold_0.25.npz
Pt_p2/pynxraw/Mask_Threshold_0.3.npz
Pt_p2/pynxraw/S1398_ampdispstrain_gap_iso0.2_mode_avg3_apodize_blackman_lab-frame.npz
Pt_p2/pynxraw/S1398_avg_obj_prtf_gap_iso0.2_mode_avg3_apodize_blackman.npz
Pt_p2/pynxraw/S1398_maskpynx_norm_128_300_294_1_1_1.npz
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1.npz


In [3]:
%run -i Scripts/PlotNPZ

Path to file from current folder: Pt_p2/pynxraw/S1398_avg_obj_prtf_gap_iso0.2_mode_avg3_apodize_blackman.npz
Stored arrays :['obj']


interactive(children=(Dropdown(description='Array used:', options=('obj',), style=DescriptionStyle(description…

In [2]:
%run -i Scripts/PlotNPZ

Path to file from current folder: Pt_p2/pynxraw/S1398_ampdispstrain_gap_iso0.2_mode_avg3_apodize_blackman_lab-frame.npz
Stored arrays :['amp', 'displacement', 'bulk', 'strain']


interactive(children=(Dropdown(description='Array used:', options=('amp', 'displacement', 'bulk', 'strain'), s…

<font color=#163e5a>  
    <h1>Find the .cxi files</h1>
    <a href = "https://raw.githubusercontent.com/cxidb/CXI/master/cxi_file_format.pdf" target = "_blank">Documentation</a>
</font>

In [13]:
%%bash
ls Pt_p2/pynxraw/*.cxi

Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-25-40_Run0008_LLKf000.1159_LLK000.1328_SupportThreshold0.29541.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-26-53_Run0010_LLKf000.1158_LLK000.1330_SupportThreshold0.29706.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-37-13_Run0027_LLKf000.1157_LLK000.1329_SupportThreshold0.29221.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-39-39_Run0031_LLKf000.1158_LLK000.1330_SupportThreshold0.28916.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-41-28_Run0034_LLKf000.1158_LLK000.1332_SupportThreshold0.28754.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-44-30_Run0039_LLKf000.1158_LLK000.1330_SupportThreshold0.29166.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-47-34_Run0044_LLKf000.1158_LLK000.1329_SupportThreshold0.29538.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-59-06_Run0063_LLKf000.1155_LLK000.1

* The last .cxi file contains only the detector and the mask data, output of PyNX

* All the other .cxi files are in real space and show the **complex** electronic density, they each are the output of the phase retrieval process. The image size in real space is also given.
* The support is linked to the mask, seems that it must be a binary array (0 masks, 1 does not mask)
* Free pixel mask is not the original .npz file ? Boolean array maybe fixes which pixels cannot be assigned to the recalculated intensity, final one ?
* entry_last is just a link to entry_1, idk why, doubles data size ? not sure since just link

<font color=#163e5a>  
    <h1>Check one of the 10 good datasets</h1>
    <p></p>
</font>

In [3]:
%run -i Scripts/PlotCXI

Path to file from current folder: Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-25-40_Run0008_LLKf000.1159_LLK000.1328_SupportThreshold0.29541.cxi
Data file architecture :

Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-25-40_Run0008_LLKf000.1159_LLK000.1328_SupportThreshold0.29541.cxi (File) ''
Last modif.: 'Fri Dec  4 19:26:17 2020'
Object Tree: 
/ (RootGroup) ''
/cxi_version (Array()) ''
/entry_last (SoftLink) -> /entry_1
/entry_1 (Group) ''
/entry_1/program_name (UnImplemented()) ''
/entry_1/start_time (UnImplemented()) ''
/entry_1/data_1 (Group) ''
/entry_1/data_1/data (SoftLink) -> /entry_1/image_1/data
/entry_1/image_1 (Group) ''
/entry_1/image_1/data (CArray(128, 300, 294), shuffle, zlib(4)) ''
/entry_1/image_1/data_space (UnImplemented()) ''
/entry_1/image_1/data_type (UnImplemented()) ''
/entry_1/image_1/image_size (Array(3,)) ''
/entry_1/image_1/mask (CArray(128, 300, 294), shuffle, zlib(4)) ''
/entry_1/image_1/support (SoftLink) -> /entry_1/im

interactive(children=(Dropdown(description='First 2 axes:', index=1, options=('xy', 'yz', 'xz'), style=Descrip…

<font color=#163e5a>  
    <h1>Try to extract a support determination from the 10 runs</h1>
    <p></p>
</font>

In [39]:
%run -i Scripts/MakeMask

Example of used files : Pt_p2\pynxraw\S1398_pynx_norm_128_300_294_1_1_1-2020*cxi
Example of folder : Pt_p2\pynxraw
Example of file name : S1398_pynx_norm_128_300_294_1_1_1-2020

Please write your folder like in the example: Pt_p2\pynxraw
Please write your file name like in the example: S1398_pynx_norm_128_300_294_1_1_1-2020

Used files: ['Pt_p2\\pynxraw\\S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-25-40_Run0008_LLKf000.1159_LLK000.1328_SupportThreshold0.29541.cxi', 'Pt_p2\\pynxraw\\S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-26-53_Run0010_LLKf000.1158_LLK000.1330_SupportThreshold0.29706.cxi', 'Pt_p2\\pynxraw\\S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-37-13_Run0027_LLKf000.1157_LLK000.1329_SupportThreshold0.29221.cxi', 'Pt_p2\\pynxraw\\S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-39-39_Run0031_LLKf000.1158_LLK000.1330_SupportThreshold0.28916.cxi', 'Pt_p2\\pynxraw\\S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-41-28_Run0034_LLKf000.1158_LLK000.1332_SupportThreshold0.28

<font color=#163e5a>
    
* Take an 'average', so at least half of the 10 good phase retrieval must have the point over the threshold to be in the support
    
* Need to smooth by convoluting with a gaussian, the edges look rough
</font>

<font color=#163e5a>

**Check the .npz files again, a mask should be here**
    
* We can compare different threshold for the mask
    
</font>

In [40]:
%%bash
ls Pt_p2/pynxraw/*.npz

Pt_p2/pynxraw/Mask_10_100_0_1.npz
Pt_p2/pynxraw/Mask_Threshold_0.15.npz
Pt_p2/pynxraw/Mask_Threshold_0.2.npz
Pt_p2/pynxraw/Mask_Threshold_0.25.npz
Pt_p2/pynxraw/Mask_Threshold_0.3.npz
Pt_p2/pynxraw/S1398_ampdispstrain_gap_iso0.2_mode_avg3_apodize_blackman_lab-frame.npz
Pt_p2/pynxraw/S1398_avg_obj_prtf_gap_iso0.2_mode_avg3_apodize_blackman.npz
Pt_p2/pynxraw/S1398_maskpynx_norm_128_300_294_1_1_1.npz
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1.npz


In [41]:
%run -i Scripts/PlotNPZ

Path to file from current folder: Pt_p2/pynxraw/Mask_Threshold_0.3.npz
Stored arrays :['mask']


interactive(children=(Dropdown(description='Array used:', options=('mask',), style=DescriptionStyle(descriptio…

In [21]:
%run -i Scripts/PlotNPZ

Path to file from current folder: Pt_p2/pynxraw/Mask_Threshold_0.2.npz
Stored arrays :['mask']


interactive(children=(Dropdown(description='Array used:', options=('mask',), style=DescriptionStyle(descriptio…

In [19]:
%run -i Scripts/PlotNPZ

Path to file from current folder: Pt_p2/pynxraw/Mask_10_100_0_1.npz
Stored arrays :['mask']


interactive(children=(Dropdown(description='Array used:', options=('mask',), style=DescriptionStyle(descriptio…

<font color=#163e5a>  
    <h1>Create modes</h1>
    <p>Only on lid01 because pynx package</p>
</font>

`python pynx-cdi-analysis.py S1398_pynx_norm_128_300_294_1_1_1-* modes=1`

<font color=#163e5a>  
    <h1>Launch strain analysis</h1>
    <p>Only on lid01 because local bcdi package has problems, the window opens, I can click on a file but then nothing happens (for Jupyter) and it is blocked. In the command, it crashed after the 3rd figure but here I do not see the figure opening.</p>
</font>

`python D:/Documents/PythonScripts/PhDScripts/Pt_p2/pynxraw/strain.py`

<font color=#163e5a>  
    <h1>Check final data</h1>

* `S1398_ampdispstrain_gap_iso0.2_mode_avg3_apodize_blackman_lab-frame.npz` is the reconstructed data in real space with detail on bulk (support), strain
* `S1398_avg_obj_prtf_gap_iso0.2_mode_avg3_apodize_blackman.npz` is the electronic density of the object
</font>

In [4]:
%%bash
ls Pt_p2/pynxraw/*.npz

Pt_p2/pynxraw/Mask_10_100_0_1.npz
Pt_p2/pynxraw/Mask_Threshold_0.15.npz
Pt_p2/pynxraw/Mask_Threshold_0.2.npz
Pt_p2/pynxraw/Mask_Threshold_0.25.npz
Pt_p2/pynxraw/Mask_Threshold_0.3.npz
Pt_p2/pynxraw/S1398_ampdispstrain_gap_iso0.2_mode_avg3_apodize_blackman_lab-frame.npz
Pt_p2/pynxraw/S1398_avg_obj_prtf_gap_iso0.2_mode_avg3_apodize_blackman.npz
Pt_p2/pynxraw/S1398_maskpynx_norm_128_300_294_1_1_1.npz
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1.npz


In [43]:
%run -i Scripts/PlotNPZ

Path to file from current folder: Pt_p2/pynxraw/S1398_ampdispstrain_gap_iso0.2_mode_avg3_apodize_blackman_lab-frame.npz
Stored arrays :['amp', 'displacement', 'bulk', 'strain']


interactive(children=(Dropdown(description='Array used:', options=('amp', 'displacement', 'bulk', 'strain'), s…